#Introduction to the A/B Testing Project
##Project Overview
This project analyzes an A/B test conducted by a UK-based foodtech company to evaluate the impact of larger food images on restaurant menu cards on user experience and conversion rates.

The experiment was designed to determine whether increasing the size of food images helps users better understand meal options, leading to higher order rates and a more seamless decision-making process.

## Key Objectives of the A/B Test
  * Test Hypothesis: Does displaying larger food images on restaurant menu cards lead to higher conversion rates (i.e., more users placing and completing orders)?
  * Measure User Behavior: Identify how users engage with restaurant pages, how long they spend before ordering, and whether larger images impact drop-off rates.
  * Compare Conversion Performance: Analyze the differences in order placements, successful orders, and drop-off points between the Control Group (original small images) and Test Group (larger images).
  * Evaluate External Influences: Consider how factors such as time of day, user platform (iOS vs. Android), and Black Friday promotions may have influenced conversion rates.
## Experiment Details
  * Location: London, UK
  * Test Duration: November 25 – November 30, 2024
  * User Groups:
  * Control Group: Saw the original smaller images.
  * Test Group: Saw larger food images.
  * Dataset Information: The dataset contains user interaction events, including restaurant visits, order placements, completed orders, and drop-offs.
## Approach & Methodology
This project follows a data-driven approach to extract meaningful insights from the A/B test. The analysis includes:
  * Data Preprocessing – Cleaning and structuring the dataset for accurate analysis.
  * Exploratory Data Analysis (EDA) – Examining user interactions, time spent per session, and drop-off points.
  * Statistical Testing – Conducting Chi-Square tests to determine whether the differences in conversion rates are statistically significant.
  * Segmented Analysis – Breaking down results by platform (iOS vs. Android), time of day (morning, afternoon, evening), and long-term user retention.
  * Business Recommendations – Providing actionable insights for the company to optimize user engagement and improve future decision-making.

## Why This Analysis Matters
Understanding the results of this A/B test is critical for improving the user experience and increasing revenue. By identifying how visual elements impact consumer behavior, this analysis helps the company:

  * Optimize UI/UX design for better engagement.
  * Reduce drop-off rates and increase successful orders.
  * Strategically time promotions based on user behavior insights.

This project aims to provide clear, data-backed recommendations on whether larger images should be permanently adopted and how the company can further refine its food ordering experience. 🚀

# Data Cleaning

In [ ]:
import pandas as pd
import plotly.express as px
from scipy.stats import chi2_contingency

In [ ]:
df = pd.read_csv('DATASET.csv')

In [ ]:
df.head(10)

,event_id,session_id,user_id,variation,platform,datetime_event,event_type,final_order_status,shop_id
16000,21112192,2994108,100003,1,iOS,2024-11-30 17:35:59,entry_to_shop,Not Successful,8272.0
212245,72673228,5048955,100009,1,Android,2024-11-29 15:57:45,entry_to_shop,Not Successful,9254.0
212246,42139926,6014379,100009,1,Android,2024-11-28 21:38:49,entry_to_shop,Not Successful,8245.0
212247,99166535,8390756,100009,1,Android,2024-11-29 14:05:24,entry_to_shop,successful,1072.0
212248,38469452,8390756,100009,1,Android,2024-11-29 14:16:49,order_paid,successful,1072.0
212249,80722654,8390756,100009,1,Android,2024-11-29 14:40:32,order_finished,successful,1072.0
212250,85739829,9365381,100009,1,Android,2024-11-28 20:47:59,entry_to_shop,Not Successful,6877.0
64950,77345911,9284425,100013,1,iOS,2024-11-28 21:07:19,reload_the_page,successful,4365.0
64951,38905109,9284425,100013,1,iOS,2024-11-28 21:07:25,entry_to_shop,successful,4365.0
64952,21181574,9284425,100013,1,iOS,2024-11-28 21:15:11,order_paid,successful,4365.0


In [ ]:
# Sort values by user and time for sequential event analysis
df.sort_values(by=['user_id', 'session_id', 'datetime_event'], inplace=True)

# Convert datetime_event to proper datetime format for time-based analysis
df['datetime_event'] = pd.to_datetime(df['datetime_event'].astype(str).str[:20], errors='coerce')

# Checking for duplicate event entries
duplicates = df.duplicated().sum()

# Summary of cleaned data
data_summary = {
    "Total number of entries in the dataset is": len(df),
    "Total number of users who participated in the survey": df.user_id.nunique(),
    "Total number of event done by the users": df.event_id.nunique(),
    "Total number of sessions": df.session_id.nunique(),
    "Total number of shop on the platform": df.shop_id.nunique(),
    "Duplicate Records": duplicates
}

# Display summary of preprocessing steps
data_summary

{'Total number of entries in the dataset is': 326921,
 'Total number of users who participated in the survey': 100000,
 'Total number of event done by the users': 326318,
 'Total number of sessions': 179294,
 'Total number of shop on the platform': 1000,
 'Duplicate Records': 0}

In [ ]:
df.head(10)

,event_id,session_id,user_id,variation,platform,datetime_event,event_type,final_order_status,shop_id
16000,21112192,2994108,100003,1,iOS,2024-11-30 17:35:59,entry_to_shop,NaN,8272.0
212245,72673228,5048955,100009,1,Android,2024-11-29 15:57:45,entry_to_shop,NaN,9254.0
212246,42139926,6014379,100009,1,Android,2024-11-28 21:38:49,entry_to_shop,NaN,8245.0
212247,99166535,8390756,100009,1,Android,2024-11-29 14:05:24,entry_to_shop,successful,1072.0
212248,38469452,8390756,100009,1,Android,2024-11-29 14:16:49,order_paid,successful,1072.0
212249,80722654,8390756,100009,1,Android,2024-11-29 14:40:32,order_finished,successful,1072.0
212250,85739829,9365381,100009,1,Android,2024-11-28 20:47:59,entry_to_shop,NaN,6877.0
64950,77345911,9284425,100013,1,iOS,2024-11-28 21:07:19,reload_the_page,successful,4365.0
64951,38905109,9284425,100013,1,iOS,2024-11-28 21:07:25,entry_to_shop,successful,4365.0
64952,21181574,9284425,100013,1,iOS,2024-11-28 21:15:11,order_paid,successful,4365.0


In [ ]:
#Checking for missing values
df.isna().sum()

,0
event_id,0
session_id,0
user_id,0
variation,0
platform,0
datetime_event,0
event_type,0
final_order_status,156289
shop_id,11079


In [ ]:
# Filling up missing shop_id values with a placeholder (-1), this would help in distinguishing missing vs. actual shop IDs
df.shop_id.fillna(-1, inplace=True)

<ipython-input-5-2de99d20bac5>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.shop_id.fillna(-1, inplace=True)


In [ ]:
df.final_order_status.unique()

array([nan, 'successful', 'refunded_after_delivery', 'cancelled'],
      dtype=object)

In [ ]:
# Fill missing final_order_status with "Not Successful" since it only applies to users who completed a purchase
df.final_order_status.fillna("No Order Made", inplace=True)

<ipython-input-7-93381f58f5c6>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.final_order_status.fillna("No Order Made", inplace=True)


In [ ]:
# Count the number of entries for each unique final_order_status
order_status_counts = df['final_order_status'].value_counts(dropna=False)

# DataFrame to better vizualize the result
order_status_summary = pd.DataFrame({
    "Final Order Status": order_status_counts.index,
    "Count": order_status_counts.values
})

order_status_counts.reset_index()

,final_order_status,count
0,No Order Made,156289
1,successful,146035
2,cancelled,17199
3,refunded_after_delivery,7398


In [ ]:
order_status_summary.columns

Index(['Final Order Status', 'Count'], dtype='object')

In [ ]:
# Count of users in each variation group
user_counts = df.groupby('variation')['user_id'].nunique().reset_index()
user_counts.columns = ['Variation', 'Unique Users']
user_counts['Variation'] = user_counts['Variation'].map({1: 'Control (Original)', 2: 'Test (Larger Images)'})

# Bar Chart showing users number of users in controlled and Testt group
fig = px.bar(user_counts, x='Variation', y='Unique Users',
             title="Number of Users in Each A/B Testing Group",
             labels={'Unique Users': 'Number of Users', 'Variation': 'Test Group'},
             color='Variation', text='Unique Users',width=700, height=700)
fig.show()

In [ ]:
# Bar Chart showing users number of users in controlled and Test group
fig = px.bar(order_status_summary, x='Final Order Status', y='Count',
             title="Number of enteries recorded for each final order status",
             labels={'Unique Users': 'Final Order Status', 'Variation': 'Count'},
             color='Final Order Status', text='Count',width=700, height=700)
fig.show()

In [ ]:
# Count occurrences of each event type
event_counts = df['event_type'].value_counts().reset_index()
event_counts.columns = ['Event Type', 'Count']

# Create a bar chart using Plotly
fig = px.bar(event_counts, x='Event Type', y='Count',
             title="Distribution of User Events",
             labels={'Count': 'Number of Events', 'Event Type': 'Event Type'},
             color='Event Type', text='Count',width=700, height=700)

# Display the interactive plot
fig.show()

In [ ]:
# Identify if there are any users who participated in both variations (1 = Control, 2 = Test)
users_in_both = df.groupby('user_id')['variation'].nunique()

# Filter for users who have been in both variations
users_in_both = users_in_both[users_in_both > 1]
num_users_in_both = len(users_in_both)
print('Total number of users who participated in both variations are :', num_users_in_both)

Total number of users who participated in both variations are : 0


#Checking for Imbalance in the Dataset
Before running an A/B test analysis, it is important to ensure that the dataset is balanced between the test and control groups. A statistical imbalance in the dataset can introduce bias, leading to misleading conclusions
## Why Check for Imbalance?
  * Fair Comparison: If one variation has significantly more users than the other, any difference in conversion rates may be due to differences in sample size rather than the actual effect of the change.
  * Statistical Validity: A well-balanced dataset ensures that randomization was properly applied when splitting users into groups.
  * Eliminating Confounding Variables: If one group has more high-intent users (e.g., more repeat customers), it could inflate conversion rates, making it seem like the variation had an impact when it actually didn't.
## How We Check for Imbalance
  * Compare the number of users in each group (Test vs. Control).
  * Compare engaged users (users who entered a shop, placed an order, or reloaded a page) in each group.
  * Run a Chi-Square test to check if the proportion of users in each group differs significantly.

If the dataset is imbalanced, the results of our A/B test could be skewed, requiring adjustments such as stratified sampling or weighting methods.

In [ ]:
# Step 1: Create contingency table for entry_to_shop → order_paid conversion (NOT statistically significant)
conversion_table_entry_to_order = df[df['event_type'] == 'order_paid'].groupby('variation')['user_id'].nunique().reset_index()
conversion_table_entry_to_order.columns = ['Variation', 'Users Placed Order']

# Get total users who entered a shop
total_entry_users = df[df['event_type'] == 'entry_to_shop'].groupby('variation')['user_id'].nunique().reset_index()
total_entry_users.columns = ['Variation', 'Total Users Entered Shop']

# Merge to create full table
conversion_table_entry_to_order = conversion_table_entry_to_order.merge(total_entry_users, on='Variation', how='left')
conversion_table_entry_to_order['Users Not Placed Order'] = conversion_table_entry_to_order['Total Users Entered Shop'] - conversion_table_entry_to_order['Users Placed Order']

# Prepare data for chi-square test
chi2_data_entry_to_order = conversion_table_entry_to_order[['Users Placed Order', 'Users Not Placed Order']].values

# Perform Chi-Square test
chi2_stat_entry, p_value_entry, dof_entry, expected_entry = chi2_contingency(chi2_data_entry_to_order)

# Step 2: Create contingency table for entry_to_shop → successful final order (STATISTICALLY SIGNIFICANT)
conversion_table_entry_to_successful = df[df['final_order_status'] == 'successful'].groupby('variation')['user_id'].nunique().reset_index()
conversion_table_entry_to_successful.columns = ['Variation', 'Users Successfully Ordered']

# Merge with total users who entered a shop
conversion_table_entry_to_successful = conversion_table_entry_to_successful.merge(total_entry_users, on='Variation', how='left')
conversion_table_entry_to_successful['Users Not Successfully Ordered'] = conversion_table_entry_to_successful['Total Users Entered Shop'] - conversion_table_entry_to_successful['Users Successfully Ordered']

# Prepare data for chi-square test
chi2_data_entry_to_successful = conversion_table_entry_to_successful[['Users Successfully Ordered', 'Users Not Successfully Ordered']].values

# Perform Chi-Square test
chi2_stat_successful, p_value_successful, dof_successful, expected_successful = chi2_contingency(chi2_data_entry_to_successful)

# Display results
chi_square_results_entry_to_order = {
    "Chi-Square Statistic": chi2_stat_entry,
    "P-Value": p_value_entry,
    "Degrees of Freedom": dof_entry,
    "Expected Frequencies": expected_entry.tolist(),
    "Statistical Significance?": "NOT significant" if p_value_entry > 0.05 else "Significant"
}

chi_square_results_entry_to_successful = {
    "Chi-Square Statistic": chi2_stat_successful,
    "P-Value": p_value_successful,
    "Degrees of Freedom": dof_successful,
    "Expected Frequencies": expected_successful.tolist(),
    "Statistical Significance?": "NOT significant" if p_value_successful > 0.05 else "Significant"
}
# Show chi-square results
chi_square_results_entry_to_order, chi_square_results_entry_to_successful


({'Chi-Square Statistic': 0.5000912115356383,
  'P-Value': 0.4794600473741103,
  'Degrees of Freedom': 1,
  'Expected Frequencies': [[27515.545034426177, 35065.45496557382],
   [14822.454965573821, 18889.545034426177]],
  'Statistical Significance?': 'NOT significant'},
 {'Chi-Square Statistic': 18.41659980840356,
  'P-Value': 1.775048932349978e-05,
  'Degrees of Freedom': 1,
  'Expected Frequencies': [[24219.892692096, 38361.107307904],
   [13047.107307904002, 20664.892692096]],
  'Statistical Significance?': 'Significant'})

### Chi-Square Test Results: Users Who Placed an Order vs. Users Who Did Not (Control vs. Test Group):

1. Entry to Shop → Order Paid (NOT Statistically Significant)
  * Key Results
      * Chi-Square Statistic: 0.50
      * P-Value: 0.479
      * Degrees of Freedom: 1

  * Expected Frequencies (if groups were balanced):
      * Control: 27,515 expected orders vs. 35,065 expected non-orders
      * Test: 14,822 expected orders vs. 18,889 expected non-orders

  * Interpretation
    * The p-value (0.479) is greater than the standard significance level (0.05).
    * This means that the difference in conversion rates from entering a shop to placing an order is NOT statistically significant.
    * The Test Group (Larger Images) did not significantly increase the likelihood of placing an order after entering a shop.
    * This suggests that larger images alone do not directly drive users to place an order—other factors like restaurant selection and pricing may have a stronger influence.

2. Entry to Shop → Successful Order (Statistically Significant)
    * Key Results
      * Chi-Square Statistic: 18.42
      * P-Value: 1.77e-05 (~0.0000177)
      * Degrees of Freedom: 1

  * Expected Frequencies (if groups were balanced):
      * Control: 24,219 expected successful orders vs. 38,361 expected non-successful orders
      * Test: 13,047 expected successful orders vs. 20,664 expected non-successful orders

  * Interpretation
    * The p-value (0.0000177) is extremely low, which is well below the standard significance level (0.05).
    * This means that the difference in conversion rates from entering a shop to completing a successful order is statistically significant.
    * The Test Group (Larger Images) had a higher rate of successful orders than expected, confirming that larger images helped reduce order cancellations and refunds.
    * This suggests that larger images improved customer confidence, leading to fewer post-purchase cancellations or refund requests.

# Order Outcomes by Variation

Here we look at order outcome by variations

In [ ]:
# Count occurrences of each final order status per variation
order_status_counts = df.groupby(['variation', 'final_order_status'])['user_id'].nunique().reset_index()
order_status_counts.columns = ['Variation', 'Final Order Status', 'User Count']
order_status_counts['Variation'] = order_status_counts['Variation'].map({1: 'Control (Original)', 2: 'Test (Larger Images)'})

# Create a properly labeled bar chart using Plotly
fig = px.bar(order_status_counts, x='Final Order Status', y='User Count', color='Variation',
             barmode='group',
             title="Order Outcomes by Variation",
             labels={'User Count': 'Number of Users', 'Final Order Status': 'Order Status', 'Variation': 'Test Group'},
             text='User Count', width = 1000, height = 700)

fig.show()

###Key Insights

1. The Test Group (Larger Images) Had Fewer Successful Orders

  * The Control Group had more successful orders compared to the Test Group.

2. More users cancelled their orders in the Control Group.
  * This suggests that users who took longer to decide (control group) may have had more second thoughts after ordering.
  * Refunds Were Much Lower in the Test Group

3. The Test Group had significantly fewer refunded orders compared to the Control Group.

# How many Session Did a User Have Before Placing a Successful Order

For the test and control group this would tell us how long it took the user before making a decision on what to order. It would help us understand their thought process, if an order was placed upon first entry to the page, or if they needed a little more convincing before placing an order.

In [ ]:
# Number of sessions by a user before making an orde
df_sessions = df.copy()
df_sessions['is_successful_order'] = df_sessions['final_order_status'] == 'successful'

user_session_counts = df_sessions.groupby(['user_id', 'variation'])['session_id'].nunique().reset_index()
user_session_counts.columns = ['user_id', 'variation', 'total_sessions']

# Number of sessions before the first successful order
successful_sessions = df_sessions[df_sessions['is_successful_order']].groupby(['user_id', 'variation'])['session_id'].nunique().reset_index()
successful_sessions.columns = ['user_id', 'variation', 'successful_sessions']

# Merging both total and successfull session count together
user_session_analysis = user_session_counts.merge(successful_sessions, on=['user_id', 'variation'], how='left').fillna(0)
user_session_analysis['unsuccessful_sessions'] = user_session_analysis['total_sessions'] - user_session_analysis['successful_sessions']

# Average sessions before a successful order
session_stats = user_session_analysis.groupby('variation').agg(
    total_users=('user_id', 'count'),
    avg_sessions_per_user=('total_sessions', 'mean'),
    avg_unsuccessful_sessions=('unsuccessful_sessions', 'mean'),
    avg_sessions_before_success=('successful_sessions', 'mean')
).reset_index()

session_stats['variation'] = session_stats['variation'].map({1: "Control Group", 2: "Test Group (Larger Images)"})

session_stats

,variation,total_users,avg_sessions_per_user,avg_unsuccessful_sessions,avg_sessions_before_success
0,Control Group,65000,1.793677,1.352738,0.440938
1,Test Group (Larger Images),35000,1.791571,1.328800,0.462771


### Key Insights

1. User Sessions Before Successful Order
  * Users in the Test Group (Larger Images) placed successful orders in fewer sessions on average than those in the Control Group.
  * The Control Group users had more unsuccessful sessions before converting, indicating more decision-making friction.

# Major Drop-off Point

In [ ]:
#Identifying major drop-off points for both group
drop_off_counts = df_sessions[df_sessions['event_type'].isin(['reload_the_page', 'entry_to_shop'])].groupby(['variation', 'event_type'])['user_id'].nunique().reset_index()
drop_off_counts.columns = ['Variation', 'Event Type', 'Users Dropped Off']
drop_off_counts['Variation'] = drop_off_counts['Variation'].map({1: "Control Group", 2: "Test Group (Larger Images)"})
drop_off_counts

drop_off_counts['Event_Label'] = drop_off_counts['Event Type'] + " - " + drop_off_counts['Variation']

# Create a bar chart using Plotly
fig = px.bar(drop_off_counts, x='Event_Label', y='Users Dropped Off',
             title="Major Drop-Off Points by Variation",
             labels={'Users Dropped Off': 'Number of Users Dropped Off', 'Event_Label': 'Event Type & Variation'},
             color='Event Type', text='Users Dropped Off', width = 1000, height = 1000)

# Display the interactive plot
fig.show()

### Key Insights
1. Drop-Off Points
  * The highest drop-off occurred at the "entry to shop" stage in both groups, meaning many users visited a shop but did not proceed to order.
  * Reloading behavior was the second biggest drop-off point, suggesting users were looking for more options or re-evaluating their choices.

2. Which Group Had a Better One-Session Purchase Rate?
  * Test Group users were more likely to complete a purchase in a single session than Control Group users.
  * This suggests that larger images help users decide faster, leading to a better purchase rate in a single session.

# How long Did a user spend on the menu before placing an order?

Reasoning: Longer time on the menu may indicate higher engagement with the menu. So the longer a user spends on the menu can tell us how engaged they are with the menu

In [ ]:
# Calculate time difference between entry_to_shop and order_paid for each user-session per variation
df_time_group = df[df['event_type'].isin(['entry_to_shop', 'order_paid'])].copy()

# Shift datetime_event to get previous event time per session per user
df_time_group['prev_event'] = df_time_group.groupby(['user_id', 'session_id'])['datetime_event'].shift(1)

# Calculate time spent before ordering
df_time_group['time_diff'] = (df_time_group['datetime_event'] - df_time_group['prev_event']).dt.total_seconds()

# Only Paid Event
df_time_group_ordered = df_time_group[df_time_group['event_type'] == 'order_paid']

# Average time spent before ordering
avg_time_per_group = df_time_group_ordered.groupby('variation')['time_diff'].mean().reset_index()
avg_time_per_group.columns = ['Variation', 'Avg Time Before Ordering (Seconds)']

# Convert to minutes
avg_time_per_group['Avg Time Before Ordering (Minutes)'] = avg_time_per_group['Avg Time Before Ordering (Seconds)'] / 60
avg_time_per_group['Variation'] = avg_time_per_group['Variation'].map({1: "Control Group", 2: "Test Group (Larger Images)"})

fig = px.bar(avg_time_per_group, x='Variation', y='Avg Time Before Ordering (Minutes)',
             title="Average Time Spent Before Placing an Order",
             labels={'Avg Time Before Ordering (Minutes)': 'Avg Time (Minutes)', 'Variation': 'Variation Group'},
             color='Variation', text='Avg Time Before Ordering (Minutes)', width = 700, height = 700)
fig.show()

### Key Insights
1. Users in the Test Group (Larger Images) Ordered Faster
  * The user in the test group spends an average of 3 minutes before placing an order.
  * The test group spent significantly less time before ordering compared to the control group.
  * This confirms that larger images help users make faster decisions.

2. Users in the Control Group Took More Time to Decide
  * Users in this group spends roughly 5 minutes before placing an order.
  * The control group users spent more time navigating and deciding before completing an order.
  * This suggests that smaller images might require users to do more evaluation before making a choice.

After Successfully Placing an Order, Which Group had the Most returing User

Reasoning: Our reasoning for this can be what I saw in the image was exactly what was delivered.

# HOW MANY TIMES DID A USER COME BACK TO PLACE AN ORDER
If a user returns to place an order does it mean they liked what they ordered, and what was ordered was what was delivered

In [ ]:
# Count the number of unique orders per user per variation
order_counts = df[df['event_type'] == 'order_paid'].groupby(['user_id', 'variation']).size().reset_index(name='order_count')

# Count users who placed multiple orders in each variation
returning_users = order_counts[order_counts['order_count'] > 1].groupby('variation')['user_id'].nunique().reset_index()
returning_users.columns = ['Variation', 'Returning Users']

# Count total unique users per variation for percentage calculation
total_users = df.groupby('variation')['user_id'].nunique().reset_index()
total_users.columns = ['Variation', 'Total Users']

# Merge data to calculate returning user percentage
returning_users = returning_users.merge(total_users, on='Variation', how='left')
returning_users['Returning User Rate (%)'] = (returning_users['Returning Users'] / returning_users['Total Users']) * 100
returning_users['Variation'] = returning_users['Variation'].map({1: "Control Group", 2: "Test Group (Larger Images)"})

returning_users

,Variation,Returning Users,Total Users,Returning User Rate (%)
0,Control Group,5346,65000,8.224615
1,Test Group (Larger Images),2861,35000,8.174286


# Analysis by Platform (iOS vs. Android)

In [ ]:
# Count total unique users per platform
platform_users = df.groupby('platform')['user_id'].nunique().reset_index()
platform_users.columns = ['Platform', 'Total Unique Users']

fig = px.pie(platform_users, names='Platform', values='Total Unique Users',
             title="User Distribution by Platform",
             hole=0.4, width=400)

# Show interactive plot
fig.show()

In [ ]:
# Count unique users who placed successful orders by platform and variation
platform_conversion = df[df['final_order_status'] == 'successful'].groupby(['variation', 'platform'])['user_id'].nunique().reset_index()
platform_conversion.columns = ['Variation', 'Platform', 'Users Successfully Ordered']

# Count total users who entered a shop by platform and variation
platform_total_entry = df[df['event_type'] == 'entry_to_shop'].groupby(['variation', 'platform'])['user_id'].nunique().reset_index()
platform_total_entry.columns = ['Variation', 'Platform', 'Total Users Entered Shop']

# Merge and calculate conversion rate
platform_conversion = platform_conversion.merge(platform_total_entry, on=['Variation', 'Platform'], how='left')
platform_conversion['Conversion Rate (%)'] = (platform_conversion['Users Successfully Ordered'] / platform_conversion['Total Users Entered Shop']) * 100
platform_conversion['Variation'] = platform_conversion['Variation'].map({1: "Control Group", 2: "Test Group (Larger Images)"})
platform_conversion['Conversion Rate (%)'] = platform_conversion['Conversion Rate (%)'].round(2)

platform_conversion


fig = px.bar(platform_conversion, x='Platform', y='Conversion Rate (%)', color='Variation',
             barmode='group',
             title="Platform-Based Conversion Rate",
             labels={'Platform': 'User Platform', 'Conversion Rate (%)': 'Conversion Rate (%)'},
             text='Conversion Rate (%)',width=700, height=500)

# Show interactive plot
fig.show()

,Variation,Platform,Users Successfully Ordered,Total Users Entered Shop,Conversion Rate (%)
0,Control Group,Android,16260,43734,37.18
1,Control Group,iOS,7650,18847,40.59
2,Test Group (Larger Images),Android,9043,23642,38.25
3,Test Group (Larger Images),iOS,4314,10070,42.84


In [ ]:
# Count users who entered a shop but did not place an order (drop-off at entry_to_shop)
drop_off_entry = df[df['event_type'] == 'entry_to_shop'].groupby(['variation', 'platform'])['user_id'].nunique().reset_index()
drop_off_entry.columns = ['Variation', 'Platform', 'Total Users Entered Shop']

# Count users who placed an order
drop_off_orders = df[df['event_type'] == 'order_paid'].groupby(['variation', 'platform'])['user_id'].nunique().reset_index()
drop_off_orders.columns = ['Variation', 'Platform', 'Users Placed Order']

# Merge and calculate drop-off rate at entry_to_shop
drop_off_rate = drop_off_entry.merge(drop_off_orders, on=['Variation', 'Platform'], how='left')
drop_off_rate['Users Placed Order'].fillna(0, inplace=True)  # Fill missing values (users who never placed an order)
drop_off_rate['Drop-Off Rate (%)'] = (1 - (drop_off_rate['Users Placed Order'] / drop_off_rate['Total Users Entered Shop'])) * 100

# Count users who placed an order but did not reach a successful order (drop-off after order_paid)
drop_off_after_order = df[(df['event_type'] == 'order_paid') & (df['final_order_status'] != 'successful')].groupby(['variation', 'platform'])['user_id'].nunique().reset_index()
drop_off_after_order.columns = ['Variation', 'Platform', 'Users Dropped After Order']

# Merge and calculate drop-off rate after order placement
drop_off_rate = drop_off_rate.merge(drop_off_after_order, on=['Variation', 'Platform'], how='left')
drop_off_rate['Users Dropped After Order'].fillna(0, inplace=True)
drop_off_rate['Drop-Off After Order (%)'] = (drop_off_rate['Users Dropped After Order'] / drop_off_rate['Users Placed Order']) * 100


<ipython-input-15-2706ceb680f0>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  drop_off_rate['Users Placed Order'].fillna(0, inplace=True)  # Fill missing values (users who never placed an order)
<ipython-input-15-2706ceb680f0>:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

,Variation,Platform,Total Users Entered Shop,Users Placed Order,Drop-Off Rate (%),Users Dropped After Order,Drop-Off After Order (%)
0,Control Group,Android,43734,18782,57.054008,3588,19.103397
1,Control Group,iOS,18847,8786,53.382501,1701,19.360346
2,Test Group (Larger Images),Android,23642,10033,57.562812,1370,13.654939
3,Test Group (Larger Images),iOS,10070,4737,52.959285,647,13.658434


In [ ]:
# Transform the data for proper visualization
drop_off_rate_melted = drop_off_rate.melt(id_vars=['Variation', 'Platform'],
                                          value_vars=['Drop-Off Rate (%)', 'Drop-Off After Order (%)'],
                                          var_name='Drop-Off Stage',
                                          value_name='Drop-Off Rate')

# Plot drop-off rates using Plotly
fig = px.bar(drop_off_rate_melted, x='Platform', y='Drop-Off Rate', color='Variation',
             barmode='group', facet_col='Drop-Off Stage',
             title="Drop-Off Rates by Platform",
             labels={'Platform': 'User Platform', 'Drop-Off Rate': 'Drop-Off Rate (%)'}, width = 1000, height = 600)

# Show interactive plot
fig.show()

# Analysis by Time of Day

In [ ]:
# Extract hour of the day from the event timestamp
df['hour'] = df['datetime_event'].dt.hour

# Count successful orders by hour and variation
time_conversion = df[df['final_order_status'] == 'successful'].groupby(['variation', 'hour'])['user_id'].nunique().reset_index()
time_conversion.columns = ['Variation', 'Hour of Day', 'Users Successfully Ordered']

# Count total users who entered a shop by hour and variation
time_total_entry = df[df['event_type'] == 'entry_to_shop'].groupby(['variation', 'hour'])['user_id'].nunique().reset_index()
time_total_entry.columns = ['Variation', 'Hour of Day', 'Total Users Entered Shop']

# Merge and calculate conversion rate
time_conversion = time_conversion.merge(time_total_entry, on=['Variation', 'Hour of Day'], how='left')
time_conversion['Conversion Rate (%)'] = (time_conversion['Users Successfully Ordered'] / time_conversion['Total Users Entered Shop']) * 100
time_conversion['Variation'] = time_conversion['Variation'].map({1: "Control Group", 2: "Test Group (Larger Images)"})

# time bins for morning, afternoon, and evening
def categorize_time(hour):
    if 5 <= hour < 12:
        return 'Morning (5 AM - 12 PM)'
    elif 12 <= hour < 17:
        return 'Afternoon (12 PM - 5 PM)'
    else:
        return 'Evening (5 PM - 5 AM)'

# Apply time binning
time_conversion['Time Period'] = time_conversion['Hour of Day'].apply(categorize_time)

# Aggregate conversion rates by time period and variation
time_conversion_binned = time_conversion.groupby(['Variation', 'Time Period'])[['Users Successfully Ordered', 'Total Users Entered Shop']].sum().reset_index()
time_conversion_binned['Conversion Rate (%)'] = (time_conversion_binned['Users Successfully Ordered'] / time_conversion_binned['Total Users Entered Shop']) * 100

# Round conversion rates to 2 decimal places
time_conversion_binned['Conversion Rate (%)'] = time_conversion_binned['Conversion Rate (%)'].round(2)

# Define a custom sorting order for time periods
time_period_order = ["Morning (5 AM - 12 PM)", "Afternoon (12 PM - 5 PM)", "Evening (5 PM - 5 AM)"]

# Convert the 'Time Period' column to a categorical type with a specified order
time_conversion_binned['Time Period'] = pd.Categorical(time_conversion_binned['Time Period'], categories=time_period_order, ordered=True)

# Sort the data based on the custom order
time_conversion_binned = time_conversion_binned.sort_values("Time Period")

# Show interactive plot for total successful orders
fig1 = px.bar(time_conversion_binned, x='Time Period', y='Users Successfully Ordered', color='Variation',
             barmode='group',
             title="Total Successful Orders by Time Period (Morning, Afternoon, Evening)",
             labels={'Time Period': 'Time of Day', 'Users Successfully Ordered': 'Total Successful Orders'},
             text='Users Successfully Ordered', width=800, height=600)

fig1.show()

# Show interactive plot for conversion rates
fig2 = px.bar(time_conversion_binned, x='Time Period', y='Conversion Rate (%)', color='Variation',
             barmode='group',
             title="Conversion Rates by Time Period",
             labels={'Time Period': 'Time of Day', 'Conversion Rate (%)': 'Conversion Rate (%)'},
             text='Conversion Rate (%)', width=800, height=600)

fig2.show()

In [ ]:
time_conversion_binned

,Variation,Time Period,Users Successfully Ordered,Total Users Entered Shop,Conversion Rate (%)
2,Control Group,Morning (5 AM - 12 PM),201,617.0,32.58
5,Test Group (Larger Images),Morning (5 AM - 12 PM),132,356.0,37.08
0,Control Group,Afternoon (12 PM - 5 PM),10958,28960.0,37.84
3,Test Group (Larger Images),Afternoon (12 PM - 5 PM),6309,15603.0,40.43
1,Control Group,Evening (5 PM - 5 AM),33685,71217.0,47.30
4,Test Group (Larger Images),Evening (5 PM - 5 AM),18507,38311.0,48.31


# Conversion Analysis

Here's the steps that would be taken in other to calculate conversion accurately
1. Categorize Users Based on Their Actions:
  * Users who entered a shop but did not place an order → (Abandoners)
  * Users who placed an order and it was successfully delivered → (Successful Conversions)
  * Users who placed an order but later canceled → (Cancellations)
  * Users who placed an order, received it, but requested a refund → (Refund Requests)

2. Categorize Sessions:
  * Sessions where users just reloaded the page
  * Sessions where users entered a shop but did not order
  * Sessions where users placed an order successfully
  * Sessions where users placed an order but later canceled or refunded
  * Analyze the Number of Visits Before Conversion:

3. How many sessions did it take for a user to place a successful order?
  * Did users who abandoned the process return in a later session?
  * Do users in the test group (larger images) require fewer sessions to convert?



In [ ]:
# Grouping by user_id, session_id, and variation to track session behavior
session_summary = df.groupby(['user_id', 'session_id', 'variation']).agg({
    'event_type': list,
    'final_order_status': 'first'
}).reset_index()

# Function to define session categories based on event types and order status
def categorize_session(row):
    event_list = row['event_type']
    order_status = row['final_order_status']

    if 'order_paid' in event_list:
        if order_status == 'successful':
            return 'Successful Order'
        elif order_status == 'cancelled':
            return 'Cancelled Order'
        elif order_status == 'refunded_after_delivery':
            return 'Refunded Order'
    elif 'entry_to_shop' in event_list and 'order_paid' not in event_list:
        return 'Shop Entry with No Order'
    elif 'reload_the_page' in event_list and 'order_paid' not in event_list:
        return 'Reloaded with No Order'
    return 'Other'

# Categorizing each sessions
session_summary['session_category'] = session_summary.apply(categorize_session, axis=1)
session_category_counts = session_summary.groupby(['variation', 'session_category']).size().reset_index(name='count')
session_category_counts['variation'] = session_category_counts['variation'].map({1: "Control Group", 2: "Test Group (Larger Images)"})

# Average number of sessions
successful_orders = session_summary[session_summary['session_category'] == 'Successful Order'].groupby('variation')['session_id'].count().reset_index()
successful_orders.columns = ['variation', 'Total Successful Orders']

total_sessions = session_summary.groupby('variation')['session_id'].count().reset_index()
total_sessions.columns = ['variation', 'Total Sessions']

# Merging so as to calculate avg sessions before conversion
session_analysis = total_sessions.merge(successful_orders, on='variation', how='left')
session_analysis['Avg Sessions Before Conversion'] = session_analysis['Total Sessions'] / session_analysis['Total Successful Orders']
session_analysis['variation'] = session_analysis['variation'].map({1: "Control Group", 2: "Test Group (Larger Images)"})


In [ ]:
fig = px.bar(session_category_counts, x='session_category', y='count', color='variation',
             title="Session Categories by Variation",
             labels={'count': 'Number of Sessions', 'session_category': 'Session Category', 'variation': 'Test Group'},
             barmode='group',
             text='count',
             width = 1000, height = 700)
fig.show()

In [ ]:
session_category_counts

,variation,session_category,count
0,Control Group,Cancelled Order,3391
1,Control Group,Refunded Order,2088
2,Control Group,Reloaded with No Order,7264
3,Control Group,Shop Entry with No Order,75185
4,Control Group,Successful Order,28661
5,Test Group (Larger Images),Cancelled Order,1910
6,Test Group (Larger Images),Refunded Order,171
7,Test Group (Larger Images),Reloaded with No Order,3815
8,Test Group (Larger Images),Shop Entry with No Order,40612
9,Test Group (Larger Images),Successful Order,16197


In [ ]:
session_summary[session_summary['session_category']=='Cancelled Order']

,user_id,session_id,variation,event_type,final_order_status,session_category
268,101152,9037183,2,"[entry_to_shop, order_paid, order_finished]",cancelled,Cancelled Order
294,101256,6876091,1,"[entry_to_shop, order_paid, order_finished]",cancelled,Cancelled Order
325,101448,8774027,2,"[entry_to_shop, order_paid, order_finished]",cancelled,Cancelled Order
489,102340,4001023,2,"[entry_to_shop, order_paid, order_finished]",cancelled,Cancelled Order
498,102374,6651290,2,"[entry_to_shop, order_paid, order_finished]",cancelled,Cancelled Order
...,...,...,...,...,...,...
179105,999034,5511920,1,"[entry_to_shop, order_paid, order_finished]",cancelled,Cancelled Order
179180,999423,1376880,2,"[entry_to_shop, order_paid, order_finished]",cancelled,Cancelled Order
179220,999611,5484512,2,"[entry_to_shop, order_paid, order_finished]",cancelled,Cancelled Order
179226,999645,7727805,2,"[entry_to_shop, order_paid, order_finished]",cancelled,Cancelled Order


## Conversion Rate
Now that we have successfully categorized sessions into successful, cancelled, refunded, and non-order sessions, we can now properly calculate conversion rate based on meaningful user interactions.


Instead of just taking total users, we should base the conversion rate on users who actually engaged with the platform (entered a shop, reloaded, or placed an order).

📌 Conversion Rate Formula for User
Conversion Rate = (Users Who Successfully Ordered / Users Who Entered a Shop at Least Once ) × 100

📌 Conversion Rate Formula for Session
Conversion Rate = (Session with A successful Order  / Total Sessions ) × 100

Where:

  * Users Who Successfully Ordered → Users with at least one "Successful Order" session.
  * Users Who Entered a Shop at Least Once → Users who performed entry_to_shop (including those who did not place an order).

In [ ]:
'''
User Based Conversion Rate
'''
#Count of Users Who Successfully Ordered
successful_users = session_summary[session_summary['session_category'] == 'Successful Order'].groupby('variation')['user_id'].nunique().reset_index()
successful_users.columns = ['variation', 'Users Who Successfully Ordered']

#Count of Users Who Engaged with the Platform (Entered Shop, Ordered, or Reloaded)
engaged_users = session_summary[session_summary['session_category'].isin([
    'Successful Order', 'Cancelled Order', 'Refunded Order', 'Shop Entry with No Order'
])].groupby('variation')['user_id'].nunique().reset_index()
engaged_users.columns = ['variation', 'Total Engaged Users']

# Calculate Conversion Rate
conversion_rate = engaged_users.merge(successful_users, on='variation', how='left')
conversion_rate['Conversion Rate (%)'] = (conversion_rate['Users Who Successfully Ordered'] / conversion_rate['Total Engaged Users']) * 100
conversion_rate['variation'] = conversion_rate['variation'].map({1: "Control Group", 2: "Test Group (Larger Images)"})


'''
Session Based Conversion Rate
'''
# Count of  Sessions with a Successful Order
successful_sessions = session_summary[session_summary['session_category'] == 'Successful Order'].groupby('variation')['session_id'].nunique().reset_index()
successful_sessions.columns = ['variation', 'Sessions with Successful Order']

#Count of Total Sessions per Variation
total_sessions = session_summary.groupby('variation')['session_id'].nunique().reset_index()
total_sessions.columns = ['variation', 'Total Sessions']

# Calculate Session-Based Conversion Rate
session_conversion_rate = total_sessions.merge(successful_sessions, on='variation', how='left')
session_conversion_rate['Session Conversion Rate (%)'] = (session_conversion_rate['Sessions with Successful Order'] / session_conversion_rate['Total Sessions']) * 100
session_conversion_rate['variation'] = session_conversion_rate['variation'].map({1: "Control Group", 2: "Test Group (Larger Images)"})

In [ ]:
#User Based Conversion Rate
conversion_rate

,variation,Total Engaged Users,Users Who Successfully Ordered,Conversion Rate (%)
0,Control Group,62581,23910,38.206484
1,Test Group (Larger Images),33712,13357,39.620907


In [ ]:
#Session Based Conversion Rate
session_conversion_rate

,variation,Total Sessions,Sessions with Successful Order,Session Conversion Rate (%)
0,Control Group,116589,28661,24.582937
1,Test Group (Larger Images),62705,16197,25.830476


### Key Insights

1. Test Group Had a Higher Conversion Rate
  * Control Group: 38.2% User conversion rate and 24.6% session conversion rate
  * Test Group: 39.6% User conversion rate and 25.8% session conversion rate
  * Test Group had a +1.4% and +1.2% respectively higher conversion rate compared to the Control Group.
  
2. Total Sessions Were Higher in the Control Group
  * Control Group had more total sessions (116,589), while the Test Group had 62,705.
  * This suggests that users in the Control Group needed more sessions before converting.

3. Test Group Users Converted Faster Per Session
  * The Test Group had a higher percentage of sessions leading to a purchase, reinforcing that larger images helped users make quicker decisions.

## Session Analysis

In [ ]:
session_analysis

,variation,Total Sessions,Total Successful Orders,Avg Sessions Before Conversion
0,Control Group,116589,28661,4.067862
1,Test Group (Larger Images),62705,16197,3.871396


### Key Insights
* Test Group (Larger Images) Required Fewer Sessions to Convert

    * Control Group: 4.07 sessions per successful order
    * Test Group: 3.87 sessions per successful order
    * This indicates that users in the Test Group (Larger Images) needed fewer sessions before making a purchase, suggesting that larger images helped users make decisions faster.